<a href="https://colab.research.google.com/github/moh2236945/Natural-language-processing/blob/master/Named_Entity_Recognition_(NER).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-jjx89qwq
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-jjx89qwq
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=35888fa1f6f2eb5994debfac75a888fb905c59370d983b7aa5a7f6bf69beca34
  Stored in directory: /tmp/pip-ephem-wheel-cache-ua3lbxq0/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [6]:
!pip install sklearn-crfsuite


     |████████████████████████████████| 757kB 4.1MB/s 


In [9]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=520520250bf7420283e50674d5ccf800dfb23a9456e85edf4c4251a144940ebe
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import LSTM,Dense,TimeDistributed,Embedding,Bidirectional
from keras.models import Model,Input
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import f1_score
from seqeval.metrics import precision_score,recall_score,f1_score,classification_report
from keras.preprocessing.text import text_to_word_sequence
import pickle

In [0]:
df=pd.read_csv('file.csv',encoding = "ISO-8859-1")
df=df.fillna(method='ffill')
class sentence(object):
    def __init__(self,df):
        self.n_sent=1
        self.df=df
        self.empty=False
        agg=lambda s:[(w,p,t) for w,p,t in zip(s['Word'].values.tolist(),
                                               s['POS'].values.tolist(),
                                               s['Tag'].values.tolist())]
        self.grouped=self.df.groupby('#Sentence#').apply(agg)
        sellf.sentence=[s for s in self.grouped]

    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [0]:
#displaying full sentence
getter=sentence(df)
sentences=[''.join([s[0] for s in sent)] for sent in getter.sentences]
sentences[0]

In [0]:
#sentence with its pos and tag.
sent = getter.get_text()

In [0]:
#Getting all the sentences in the dataset.
sentences = getter.sentences

In [0]:
#Defining the parameters for LSTM network
# Number of data points passed in each iteration
batch_size = 64 
# Passes through entire dataset
epochs = 8
# Maximum length of review
max_len = 75 
# Dimension of embedding vector
embedding = 40

In [0]:
#Getting unique words and labels from data
words = list(df['Word'].unique())
tags = list(df['Tag'].unique())
# Dictionary word:index pair
# word is key and its value is corresponding index
word_to_index = {w : i + 2 for i, w in enumerate(words)}
word_to_index["UNK"] = 1
word_to_index["PAD"] = 0

# Dictionary lable:index pair
# label is key and value is index.
tag_to_index = {t : i + 1 for i, t in enumerate(tags)}
tag_to_index["PAD"] = 0

idx2word = {i: w for w, i in word_to_index.items()}
idx2tag = {i: w for w, i in tag_to_index.items()}

In [0]:
# Converting each sentence into list of index from list of tokens
X = [[word_to_index[w[0]] for w in s] for s in sentences]

# Padding each sequence to have same length  of each word
X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])
# Convert label to index
y = [[tag_to_index[w[2]] for w in s] for s in sentences]

# padding
y = pad_sequences(maxlen = max_len, sequences = y, padding = "post", value = tag_to_index["PAD"])
num_tag = df['Tag'].nunique()
# One hot encoded labels
y = [to_categorical(i, num_classes = num_tag + 1) for i in y]
In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

In [0]:
#Bidirectional LSTM-CRF Network
num_tags = df['Tag'].nunique()
# Model architecture
input = Input(shape = (max_len,))
model = Embedding(input_dim = len(words) + 2, output_dim = embedding, input_length = max_len, mask_zero = True)(input)
model = Bidirectional(LSTM(units = 50, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(50, activation="relu"))(model)
crf = CRF(num_tags+1)  # CRF layer
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

In [0]:
#Making Checkpoint each epoch to check and save the best model performance till last and also avoiding further
# validation loss drop due to overfitting.
checkpointer = ModelCheckpoint(filepath = 'model.h5',
                       verbose = 0,
                       mode = 'auto',
                       save_best_only = True,
                       monitor='val_loss')
history = model.fit(X_train, np.array(y_train), batch_size=batch_size, epochs=epochs,
                    validation_split=0.1, callbacks=[checkpointer])

In [0]:
history.history.keys()


In [0]:
# Evaluation
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=-1)
y_test_true = np.argmax(y_test, -1)
# Convert the index to tag
y_pred = [[idx2tag[i] for i in row] for row in y_pred]
print("F1-score is : {:.1%}".format(f1_score(y_test_true, y_pred)))
report = flat_classification_report(y_pred=y_pred, y_true=y_test_true)
print(report)

In [0]:
#Save the result

with open('word_to_index.pickle', 'wb') as f:
    pickle.dump(word_to_index, f)

with open('tag_to_index.pickle', 'wb') as f:
    pickle.dump(tag_to_index, f)
